In [42]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import json

# process character

In [81]:
char_processed = pd.read_csv(
    "/home/sevashasla/epfl/sem1/ada-2024-project-cyberbab00shka/data/MovieSummaries/character_processed.csv",
    parse_dates=["actor_date_of_birth", "movie_release_date"],
)

In [85]:
char_processed.isna().sum(axis=0) / char_processed.shape[0]

Wikipedia movie ID                 0.000000
Freebase movie ID                  0.000000
Character name                     0.557159
Actor gender                       0.006152
Actor height (in meters)           0.553138
Actor ethnicity (Freebase ID)      0.705642
Actor name                         0.000039
Freebase character/actor map ID    0.000000
Freebase character ID              0.557147
Freebase actor ID                  0.000000
actor_date_of_birth                0.000000
movie_release_date                 0.000000
ethn_name                          0.742623
race                               0.742623
dtype: float64

As we can see there are a lot of missing values:
- Character name                     ~55%
- Actor height (in meters)           ~55%
- Freebase character ID              ~56% -> not very useful so it is okay
- Actor ethnicity (Freebase ID)      ~70%
- ethn_name                          ~74%
- race                               ~74%

In [87]:
char_processed.isna().sum(axis=1).value_counts()

6    103185
4     54281
3     53046
5     44665
0     40255
2     26844
1     11133
7      1577
8         1
Name: count, dtype: int64

In [88]:
char_processed[char_processed.isna().sum(axis=1) == 6]

,Wikipedia movie ID,Freebase movie ID,Character name,Actor gender,Actor height (in meters),Actor ethnicity (Freebase ID),Actor name,Freebase character/actor map ID,Freebase character ID,Freebase actor ID,actor_date_of_birth,movie_release_date,ethn_name,race
12,975900,/m/03vyhn,NaN,F,NaN,NaN,Rosemary Forsyth,/m/0cg7d4h,NaN,/m/03wc_yl,1943-07-06,2001-08-24,NaN,NaN
16,3196793,/m/08yl5d,NaN,M,NaN,NaN,Gary Dubin,/m/0lr37f2,NaN,/m/02pnjkh,1959-05-05,2000-02-16,NaN,NaN
19,3196793,/m/08yl5d,NaN,M,NaN,NaN,Brad Greenquist,/m/0lr37dm,NaN,/m/0k1vd2,1959-10-08,2000-02-16,NaN,NaN
21,3196793,/m/08yl5d,NaN,M,NaN,NaN,Chris Krisea,/m/0lr37fr,NaN,/m/0gcdnn4,1949-01-01,2000-02-16,NaN,NaN
22,28463795,/m/0crgdbh,NaN,M,NaN,NaN,Frank Krog,/m/0gct1bn,NaN,/m/053j7xf,1954-10-05,1988-01-01,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334945,15567100,/m/03mfb7j,NaN,M,NaN,NaN,Art Acord,/m/040nz49,NaN,/m/03s2mh,1890-04-17,1922-09-11,NaN,NaN
334956,31627556,/m/0gmdz8p,NaN,M,NaN,NaN,Jhake Angelo Vargas,/m/0h1k12_,NaN,/m/0b6fhrg,1992-07-09,2011-08-24,NaN,NaN
334967,7223293,/m/0kv0xs,NaN,M,NaN,NaN,Sid Smith,/m/02vcw0s,NaN,/m/02qrq5t,1894-02-28,1925-01-01,NaN,NaN
334968,9971909,/m/02pygw1,NaN,M,NaN,NaN,Rich Little,/m/02vbk4r,NaN,/m/03m6t5,1938-11-26,1972-09-22,NaN,NaN


In [77]:
pd.to_datetime(char_processed["actor_date_of_birth"].iloc[26664]).strftime("%Y-%m-%d")

'1969-11-20'

In [74]:
char_processed["actor_date_of_birth"].iloc[0]

str

In [83]:
movies_processed = pd.read_csv(
    "/home/sevashasla/epfl/sem1/ada-2024-project-cyberbab00shka/data/MovieSummaries/movie_processed.csv",
    parse_dates=["movie_release_date"],
)

In [90]:
movie_raw = pd.read_csv(
    "/home/sevashasla/epfl/sem1/ada-2024-project-cyberbab00shka/data/MovieSummaries/movie.metadata.tsv", 
    sep='\t', 
    header=None
)

movie_raw.rename(columns={
    0: 'Wikipedia movie ID',
    1: 'Freebase movie ID',
    2: 'Movie name',
    3: 'Movie release date',
    4: 'Movie box office revenue',
    5: 'Movie runtime',
    6: 'Movie languages',
    7: 'Movie countries',
    8: 'Movie genres'
}, inplace=True)

In [115]:
all_plots_wikiids = set()

with open("/home/sevashasla/epfl/sem1/ada-2024-project-cyberbab00shka/data/MovieSummaries/plot_summaries.txt") as f:
    lines = f.read().splitlines()
    for line in lines:
        wikiid, plot_summ = line.split("\t")
        if len(plot_summ) > 0:
            all_plots_wikiids.add(int(wikiid))

len(set(movie_raw["Wikipedia movie ID"].values.tolist()) - all_plots_wikiids)

39534

In [116]:
39534 / len(movie_raw)

0.4836495761001211

In [117]:
len(movie_raw)

81741

In [104]:
line.split("\t")

['20532852',
 'A line of people  drool at the window of the shop of market butcher Buzz Buzzard. A short series of gags ensues about how Buzz dishonestly  "jacks" up all his prices. Since Woody is broke as usual, he sneaks in and gets thrown out by Buzz. On the way out, Woody collides with a bottle of invisible ink and turns partially invisible. Buzz can only see parts of Woody\'s body, and, in a somewhat gruesome scene, thinks that he\'s been dismembered, so he sweeps him into a trap door to get rid of him. When Woody awakes, he realizes what is happening, and he douses himself with the rest of the ink in order to pose as a ghost.']

In [100]:
movie_raw.shape[0]

81741

In [94]:
lines[0].split('\t')[0]

'23890098'

We almost don't have the box office revenue!
- Movie box office revenue    ~90%
- Movie runtime               ~25%
- movie_release_date          ~8%

In [89]:
movies_processed.isna().sum(axis=0) / movies_processed.shape[0]

Wikipedia movie ID          0.000000
Freebase movie ID           0.000000
Movie name                  0.000000
Movie box office revenue    0.897224
Movie runtime               0.250180
languages                   0.000000
movie_release_date          0.084437
countries_old               0.000000
countries                   0.000000
genres                      0.000000
dtype: float64